In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../src/utils")
from utils import SimpleGraphVoltDatasetLoader_Lazy
from torch_geometric_temporal.signal import temporal_signal_split
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import A3TGCN
from tqdm import tqdm


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [75]:
trafo_id = "T1330"
epochs = 25
num_timesteps_in = 12
num_timesteps_out = 4
train_ratio = 0.7
test_ratio_vs_eval_ratio = 0.5
learning_rate = 0.00001
device_str = 'cpu'

#----------------------
if device_str == 'cpu':
    torch.cuda.empty_cache()

#get dateime string of now
now = pd.Timestamp.now().strftime("%Y%m%d%H%M%S")

In [4]:
print("Loading data...")
loader = SimpleGraphVoltDatasetLoader_Lazy(trafo_id, num_timesteps_in, num_timesteps_out)
loader_data_index = loader.snapshot_index

train_dataset, test_eval_dataset = loader.temporal_signal_split_lazy(loader_data_index, train_ratio=train_ratio)
test_dataset, eval_dataset = loader.temporal_signal_split_lazy(test_eval_dataset, train_ratio=test_ratio_vs_eval_ratio)

Loading data...
Voltage index: 5


In [111]:
class TemporalGNN(torch.nn.Module):
    def __init__(self, node_features, edge_features, periods):
        super(TemporalGNN, self).__init__()
        # Attention Temporal Graph Convolutional Cell
        out_channels = 32
        self.tgnn = A3TGCN(in_channels=node_features, 
                           out_channels=out_channels, 
                           periods=periods)
        
        self.feature_linear = torch.nn.Linear(node_features, node_features)

        self.edge_features_linear = torch.nn.Linear(edge_features, 1)

        # Equals single-shot prediction
        self.linear = torch.nn.Linear(out_channels,periods)

    def forward(self, x, edge_index, edge_features):
        """
        x = Node features for T time steps
        edge_index = Graph edge indices
        edge_weights = Graph edge weights
        """

        x = x.permute(2, 0, 1)
        x = self.feature_linear(x)
        x = x.permute(1, 2, 0)

        #print(edge_features)

        #print("edge_features is none: ",torch.isnan(edge_features).any())
        edge_features = self.edge_features_linear(edge_features).squeeze()
        edge_features = F.leaky_relu(edge_features)
        #print(edge_features)

        #edge_features = torch.tensor(np.zeros(114).astype(np.float32))

        #print("edge_features is none: ",torch.isnan(edge_features).any())
        h = self.tgnn(x, edge_index,edge_weight=edge_features)
        #print("h is none: ",torch.isnan(h).any())
        h = F.relu(h)
        h = self.linear(h)
        return h


In [112]:
def train_test(model,device, train_dataset, test_dataset, optimizer, loss_fn, epochs, now):
    """
    Definition of the training loop.
    """
    epoch_losses_train = []
    epoch_losses_test = []
    
    for epoch in range(epochs):
        model.train()
        epoch_loss_train = 0

        subset = 1000
        step=0

        for snapshot_i in tqdm(train_dataset, desc="Training epoch {}".format(epoch)):
            snapshot = loader.get_snapshot(snapshot_i)
            snapshot.to(device)
            optimizer.zero_grad()

            #print(snapshot.edge_attr.shape)

            #print(f"Weights before the model: {model.state_dict()['edge_features_linear.weight']}")

            out = model(snapshot.x, snapshot.edge_index,snapshot.edge_attr)

            #print(out)
            #print(snapshot.y)
            loss = loss_fn()(out, snapshot.y)
            
            loss.backward()

            # for name, param in model.named_parameters():
            #     if param.grad is not None:
            #         print(f'Parameter: {name}, Gradient: {param.grad.norm().item()}')


            optimizer.step()
            epoch_loss_train += loss.detach().cpu().numpy()

            #print(f"Weights after the model: {model.state_dict()['edge_features_linear.weight']}")

            step+=1

            if step > subset:
                break


        epoch_losses_train.append(epoch_loss_train)

        #print(f"Epoch {epoch + 1}, FeatureMLP Weights: {model.state_dict()}")

        model.eval()
        epoch_loss_test = 0
        with torch.no_grad():

            subset = 1000
            step=0

            for snapshot_j in tqdm(test_dataset, desc="Testing epoch {}".format(epoch)):
                snapshot = loader.get_snapshot(snapshot_j)
                snapshot.to(device)
                out = model(snapshot.x, snapshot.edge_index,snapshot.edge_attr)
                loss = loss_fn()(out, snapshot.y).cpu().numpy()
                epoch_loss_test += loss

                step+=1
                if step > subset:
                    break

            epoch_losses_test.append(epoch_loss_test)
            if min(epoch_losses_test) == epoch_loss_test:
                torch.save(model.state_dict(), f"../models/A3TGCN_{now}_{trafo_id}_epochs-{epochs}_in-{num_timesteps_in}_out-{num_timesteps_out}_train-ratio-{train_ratio}_lr-{learning_rate}.pt")
            print("Epoch: {}, Train Loss: {:.7f}, Test Loss: {:.7f}".format(epoch, epoch_loss_train, epoch_loss_test))
        
        
    return epoch_losses_train, epoch_losses_test
            

In [113]:
def eval(model, loader, eval_dataset, device, loss_fn, std):
    with torch.no_grad():
        model.eval()
        loss_all = 0
        loss_elementwise = 0
        
        steps = 0
        for snapshot_i in tqdm(eval_dataset, desc="Evaluating"):
            snapshot = loader.get_snapshot(snapshot_i)
            steps += 1
            snapshot.to(device)
            out = model(snapshot.x, snapshot.edge_index,snapshot.edge_attr)
            loss_all += loss_fn()(out, snapshot.y).cpu().numpy()
            loss_elementwise += loss_fn(reduction="none")(out, snapshot.y).cpu().numpy()
        loss_all *= std/steps
        loss_elementwise *= std/steps
    return loss_all, loss_elementwise

In [114]:
print("Running training...")

#print(loader.edge_features.shape)

device = torch.device(device_str)
model = TemporalGNN(node_features=loader.num_features, edge_features=loader.num_edge_features ,periods=num_timesteps_out).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.MSELoss
losses = train_test(model, device, train_dataset, test_dataset, optimizer, loss_fn, epochs=epochs, now=now)

Running training...


Testing epoch 0:   2%|▏         | 203/10510 [00:01<01:06, 154.95it/s]


KeyboardInterrupt: 

In [ ]:
print(losses)

std = loader.mean_and_std["measurements"][1]["voltage"]

#read saved model
model.load_state_dict(torch.load(f"../models/A3TGCN_{now}_{trafo_id}_epochs-{epochs}_in-{num_timesteps_in}_out-{num_timesteps_out}_train-ratio-{train_ratio}_lr-{learning_rate}.pt"))

loss_all, loss_elementwise = eval(model, eval_dataset, device, loss_fn, std)

print("Loss all: {:.7f}".format(loss_all))
print("Loss elementwise: {}".format(loss_elementwise))

([1.1491339802742004, 0.7666801363229752, 0.665936604142189, 0.6760101169347763, 0.6738705188035965, 0.6569817215204239, 0.6447193175554276, 0.619110494852066, 0.6045716106891632, 0.5870202779769897, 0.5645135194063187, 0.5471399575471878, 0.5253028124570847, 0.5117951482534409, 0.494486927986145, 0.48034705221652985, 0.45149946957826614, 0.4471800997853279, 0.4195747599005699, 0.41423942148685455, 0.4129945859313011, 0.4715384468436241, 0.4270767644047737, 0.37174510210752487, 0.3610764741897583], [60.501656889915466, 60.48480212688446, 61.80046832561493, 61.77976679801941, 61.15756642818451, 61.082333981990814, 59.825620263814926, 59.6981517970562, 58.834627121686935, 58.34415066242218, 57.8893159031868, 57.46279412508011, 57.2630900144577, 57.213869363069534, 57.16407111287117, 57.087400048971176, 57.30658462643623, 57.152671068906784, 57.01317685842514, 57.13613063097, 57.27414157986641, 56.79776504635811, 57.93151727318764, 57.5799278318882, 57.72479897737503])


FileNotFoundError: [Errno 2] No such file or directory: '../models/A3TGCN_20231214110502_T1330_epochs-25_in-12_out-4_train-ratio-0.7_lr-0.01.pt'